## Imports

In [14]:
from facenet_pytorch import InceptionResnetV1
from mtcnn import MTCNN
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import torch
import time

%matplotlib inline

## Iniciando os modelos e o DataFrame

In [3]:
# Inicialize o modelo MTCNN
detector = MTCNN()

# Carregue o modelo FaceNet (InceptionResnetV1 do facenet-pytorch)
model = InceptionResnetV1(pretrained='vggface2').eval()

# Crie um DataFrame para armazenar as imagens
df_images = pd.DataFrame(columns=['nome','image_path', 'features'])

## Capturando 5 Fotos, 1 Foto a cada 3 segundos, para que possamos usa-las no reconhecimento

In [40]:
nome = input('Seu Nome: ')
cap = cv2.VideoCapture(1)  # Captura a partir da câmera (0) ou de um arquivo de vídeo
image_count = 0
capturing = False

while image_count < 5:
    ret, frame = cap.read()  # Captura um quadro da câmera
    if not ret:
        break

    if not capturing:
        cv2.imshow('Câmera', frame)
    else:
        # Realize a detecção de faces na imagem
        faces = detector.detect_faces(frame)

        for face in faces:
            x, y, width, height = face['box']

            # Desenhe uma caixa delimitadora em torno da face
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)

        cv2.imshow('Detecção de Faces', frame)

    # Pressione 'c' para iniciar a captura de imagens
    key = cv2.waitKey(1)
    if key == ord('c') and not capturing:
        capturing = True
        print("Captura de imagens iniciada")

    # Se a captura estiver ativada e o número de imagens capturadas for menor que 5, capture a imagem a cada 3 segundos
    if capturing and image_count < 5:
        if len(faces) > 0:
            image_count += 1
            print(f"{nome}_{image_count} capturada")
            # Salve a imagem com um nome exclusivo, se desejar
            cv2.imwrite(f'{nome}_{image_count}.jpg', frame)
            time.sleep(3)
        else:
            print("Nenhuma face detectada.")

    # Pressione 'q' para sair
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Carregue as 5 imagens capturadas anteriormente
image_paths = [f'{nome}_1.jpg', f'{nome}_2.jpg', f'{nome}_3.jpg', f'{nome}_4.jpg', f'{nome}_5.jpg']

for image_path in image_paths:
    img = cv2.imread(image_path)
    faces = detector.detect_faces(img)

    if faces:
        x, y, width, height = faces[0]['box']
        face = img[y:y+height, x:x+width]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)  # Converte BGR para RGB
        face = cv2.resize(face, (160, 160))  # Redimensione para 160x160
        face = np.transpose(face, (2, 0, 1))  # Transponha as dimensões
        face = torch.tensor(face, dtype=torch.float32).unsqueeze(0)  # Converta para float32
        features = model(face)
        image_name = image_path.split("_")[0]
        
        df_temp = pd.DataFrame([[image_name,image_path,features]],columns=['nome','image_path','features'])

        # Adicione a imagem e suas características ao DataFrame
        df_images = pd.concat([df_temp,df_images,])

davi
Captura de imagens iniciada
davi_1 capturada
1/1 [==============================] - 0s 29ms/step
davi_2 capturada
1/1 [==============================] - 0s 27ms/step
davi_3 capturada
1/1 [==============================] - 0s 28ms/step
davi_4 capturada
1/1 [==============================] - 0s 25ms/step
davi_5 capturada
1/1 [==============================] - 0s 27ms/step


## Reconhecimento Facial

In [42]:
cap = cv2.VideoCapture(1)  # Captura a partir da câmera (0) ou de um arquivo de vídeo

# Aguarda a tecla "c" ser pressionada para iniciar o reconhecimento
start_recognition = False

while True:
    ret, frame = cap.read()  # Captura um quadro da câmera
    if not ret:
        break

    if not start_recognition:
        cv2.putText(frame, "Pressione 'c' para iniciar o reconhecimento", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Reconhecimento Facial em Tempo Real', frame)
    else:
        faces = detector.detect_faces(frame)

        for face in faces:
            x, y, width, height = face['box']
            face_img = frame[y:y+height, x:x+width]
            face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)  # Converte BGR para RGB
            face_img = cv2.resize(face_img, (160, 160))  # Redimensione para 160x160
            face_img = np.transpose(face_img, (2, 0, 1))  # Transponha as dimensões
            face_img = torch.tensor(face_img, dtype=torch.float32).unsqueeze(0)  # Converta para float32
            features = model(face_img)

            # Realize a comparação com as características das imagens conhecidas
            recognized_image = None
            for index, row in df_images.iterrows():
                distance = torch.dist(features, row['features'])
                if distance.item() < 0.2:  # Defina uma distância limite adequada
                    recognized_image = row['nome']
                    break

            if recognized_image:
                cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
                cv2.putText(frame, f"Rosto Reconhecido: {recognized_image}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 0, 255), 2)
                cv2.putText(frame, "Rosto Nao Reconhecido", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        cv2.imshow('Reconhecimento Facial em Tempo Real', frame)

    key = cv2.waitKey(1)

    if key & 0xFF == ord('q'):
        break
    elif key & 0xFF == ord('c'):
        start_recognition = True

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 29ms/step
